<a href="https://colab.research.google.com/github/VedantBatham/Intent-Slot-Based-Attendance-Query-Assistant/blob/main/Intent_SlotFilling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import re
import json
import pandas as pd

# Sample dataset
data = {
    "Date": ["2023-10-03", "2023-10-03", "2023-10-04", "2023-10-04"],
    "Class": ["A", "B", "A", "B"],
    "Present": [45, 42, 47, 40],
    "Total": [50, 50, 50, 50]
}

df = pd.DataFrame(data)
df.head()

,Date,Class,Present,Total
0,2023-10-03,A,45,50
1,2023-10-03,B,42,50
2,2023-10-04,A,47,50
3,2023-10-04,B,40,50


In [2]:
def classify_intent(query: str) -> str:
    query = query.lower()
    if "attendance" in query:
        return "attendance_query"
    else:
        return "unknown"

In [3]:
def extract_slots(query: str):
    slots = {}

    # Extract class (e.g., Class A, Class B)
    match_class = re.search(r"class\s*([a-zA-Z])", query, re.IGNORECASE)
    if match_class:
        slots["Class"] = match_class.group(1).upper()

    # Extract date (support formats like "3rd Oct", "Oct 3", "2023-10-03")
    months = {
        "jan": "01", "feb": "02", "mar": "03", "apr": "04", "may": "05", "jun": "06",
        "jul": "07", "aug": "08", "sep": "09", "oct": "10", "nov": "11", "dec": "12"
    }
    match_date = re.search(r"(\d{1,2})(?:st|nd|rd|th)?\s*([a-zA-Z]+)", query)
    if match_date:
        day = match_date.group(1).zfill(2)
        month_str = match_date.group(2)[:3].lower()
        if month_str in months:
            slots["Date"] = f"2023-{months[month_str]}-{day}"

    match_iso = re.search(r"\d{4}-\d{2}-\d{2}", query)
    if match_iso:
        slots["Date"] = match_iso.group(0)

    return slots

In [6]:
def query_attendance(query: str):
    intent = classify_intent(query)
    if intent != "attendance_query":
        return {"error": "Unsupported query"}

    slots = extract_slots(query)
    if "Class" not in slots or "Date" not in slots:
        return {"error": "Missing slots (Class or Date)"}

    result = df[(df["Class"] == slots["Class"]) & (df["Date"] == slots["Date"])]

    if result.empty:
        return {"error": "No records found"}

    record = result.iloc[0].to_dict()
    return json.loads(json.dumps(record))

In [7]:
user_query = "Show me attendance of Class B on 3rd Oct"
query_attendance(user_query)

{'Date': '2023-10-03', 'Class': 'B', 'Present': 42, 'Total': 50}